In [5]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, avg, col,lit,call_udf,countDistinct,sproc,udf
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType
import pandas as pd

import sys
import json
import platform
import os,requests
from pathlib import Path
import glob

from configs.config import snowflake_conn_prop_local as snowflake_conn_prop

from snowflake.snowpark import version
print(version.VERSION)
session = Session.builder.configs(snowflake_conn_prop).create()


(0, 7, 0)


In [6]:
import pandas as pd

import sys
import json
import platform
import os,requests
from pathlib import Path
import glob

from configs.config import snowflake_conn_prop_local as snowflake_conn_prop
        
from src.DataValidationContext import GEDataValidationContext
from src.BatchRequest import getBatchRequest 
from src.Expectations import  createExpectationSuite, createExpectations
from src.RunLoadExpectations import runExpectaionValidation,loadValidationToDB

datasourcename='taxi_pandas_dataframe'
ge=GEDataValidationContext(datasourcename)
context=ge.getContext()
context


    

/Users/praj/Documents/Personal/learn/GreatExpectation/testpandas/great_expectations/datasource/data_connector/runtime_data_connector.py:133: DeprecationWarning: Specifying batch_identifiers as part of the RuntimeDataConnector config is deprecated as of v0.15.1 and will be removed by v0.18. Please configure batch_identifiers as part of Assets instead.
  warnings.warn(


{
  "anonymous_usage_statistics": {
    "explicit_id": true,
    "data_context_id": "f84266ef-6a12-4252-b185-ba314850f614",
    "usage_statistics_url": "https://stats.greatexpectations.io/great_expectations/v1/usage_statistics",
    "enabled": true,
    "explicit_url": false
  },
  "checkpoint_store_name": "checkpoint_store",
  "config_version": 3,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/",
        "root_directory": "/tmp/great_expectation"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "datasources": {
    "dataframe_datasource": {
      "module_name": "great_expectations.datasource",
      "batch_kwargs_generators": {
        "subdir_reader": {
          "class_name": "SubdirReaderBatchKwargsGenerator",
          "ba

In [7]:
# from great_expectations.data_context import BaseDataContext
# from great_expectations.data_context.types.base import DataContextConfig, DatasourceConfig, FilesystemStoreBackendDefaults
# from great_expectations.core.batch import BatchRequest, RuntimeBatchRequest
# from src.DataValidationContext import GEDataValidationContext

# def getBatchRequest(datasourcename,pandasDF):

#     context=GEDataValidationContext(datasourcename)
#     # print(context.getContext())

#     #Creating the batch request which will be used while running the checkpoint
#     batch_request = RuntimeBatchRequest(
#                                 datasource_name=datasourcename,
#                                     data_connector_name="default_runtime_data_connector_name",
#                                     data_asset_name="PandasData",  # This can be anything that identifies this data_asset for you
#                                     runtime_parameters={"batch_data": pandasDF},  # df is your dataframe, you have created above.
#                                     batch_identifiers={"default_identifier_name": f'default_identifier_{datasourcename}'},
#                                     )
#     return batch_request



In [8]:
# df=''
# datasourcename='taxi'
# batch=getBatchRequest(datasourcename,df)
# batch

In [9]:

pd_df=session.sql("select top 2000 * from TAXI_TRIPS_MAT_VIEW").to_pandas()



In [10]:
local_batch_request=getBatchRequest(context,datasourcename,pd_df)

In [11]:
local_batch_request

{
  "datasource_name": "taxi_pandas_dataframe",
  "data_connector_name": "default_runtime_data_connector_name",
  "data_asset_name": "PandasData",
  "runtime_parameters": {
    "batch_data": "<class 'pandas.core.frame.DataFrame'>"
  },
  "batch_identifiers": {
    "default_identifier_name": "default_identifier_taxi_pandas_dataframe"
  }
}

In [12]:
createExpectationSuite(context,"demoExp")

In [13]:
# context.create_expectation_suite(
#     expectation_suite_name='test_exp', overwrite_existing=True)

In [14]:
# createExpectations(context,suitename,local_batch_request,pandasdataframe):
createExpectations(context,"demoExp",local_batch_request,pd_df)

Calculating Metrics: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 382.05it/s]


In [15]:
res=runExpectaionValidation(context,"PandasCheckpoint",local_batch_request,"demoExp",datasourcename)

Calculating Metrics: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 317.92it/s]


In [16]:
loadValidationToDB(session,res,"TestLoadGEValidation")

 Loaded Validation Results...
